In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

import pyfrac
from pyfrac.utils.sofiutils import *
from pyfrac.utils.modellingutils import *
from pyfrac.hydrafrac.acquisition import recgeom

In [2]:
# 3 layer model - USER DEFINED
expname = 'StySamp'
outpath = '../../FDModelling/StylolitesSample/'

# Rock Properties - subsurface models

In [3]:
xlim = 0.2  # 20cm
ylim = 0.1  # 10cm
zlim = 0.1  # 10cm


# time params (in seconds!)
dt = 5e-8
tdur = 0.0005

In [4]:
dx = dy = dz = 0.01  # 5mm

nx = int(xlim/dx)
ny = int(ylim/dy)
nz = int(zlim/dz)

# Subsurface properties
vp = 5260  # m/s
vs = 3260  # m/s
rho = 2590;  # kg/m3
 
print(nx,ny,nz)

20 10 10


In [5]:
# MAKE MODELS
mod_vp = vp * np.ones([nx, ny, nz])
mod_vs = vs * np.ones([nx, ny, nz])
mod_rho = rho * np.ones([nx, ny, nz])

In [6]:
# Write as binary
os.makedirs(os.path.join(outpath,'inputs/model/'), exist_ok=True)
mod_vp.astype(np.float32).tofile(os.path.join(outpath,'inputs/model/%s.vp'%expname))#,format='<f')
mod_vs.astype(np.float32).tofile(os.path.join(outpath,'inputs/model/%s.vs'%expname))  #,format='<f')
mod_rho.astype(np.float32).tofile(os.path.join(outpath,'inputs/model/%s.rho'%expname))  #,format='<f')

# Acquisition Geometry - Receivers & Source

In [7]:
# Source 
sx, sy, sz = 0.1, 0.05, 0.05
td = dt*5  # ignition time 5 timesteps in
fc = 10000  # central frequency
amp = 1.  # amplitude
string_list = [str(i) for i in [sx, sz, sy, td, fc, amp]]
src_string = '\t'.join(string_list)
print(src_string)

text_file = open(os.path.join(outpath,'inputs/sources.dat'), "wt")
text_file.writelines(src_string)
text_file.close()

0.1	0.05	0.05	2.5e-07	10000	1.0


In [8]:
# RECEIVERS
recA = np.array([50, 50, 0])/1000
recB = np.array([100, 50, 0])/1000
recC = np.array([150, 50, 0])/1000

recD = np.array([50, 50, 100])/1000
recE = np.array([150, 50, 100])/1000

recF = np.array([200, 50, 50])/1000
recG = np.array([0, 50, 50])/1000

recH = np.array([50, 0, 50])/1000
recI = np.array([100, 0, 50])/1000
recJ = np.array([150, 0, 50])/1000

recK = np.array([50, 100, 50])/1000
recL = np.array([100, 100, 50])/1000
recM = np.array([150, 100, 50])/1000



recs=np.vstack([recA,recB,recC,recD,
                recE,recF,recG,recH,
                recI,recJ,recK,recL, 
                recM
               ])


np.savetxt(os.path.join(outpath,'inputs/receiver.dat'),
           recs.astype(np.float32), 
           fmt='%.4f')

# BUILD JSON FOR SOFI

In [9]:
# mod params
n_xzy = mod_rho.transpose(0,2,1).shape
d_xzy = [dx, dz, dy]


tsnap_params = [dt, 1000*dt, dt*2]
smgrm_ndt = 100

SOFIjsonfilename = os.path.join(outpath,'SOFI3D_%s.json'%expname)

sofimaster = '~/Documents/Projects/MicroseismicModelling/SOFI3D-master/bin/'
num_procs = 8
sofi_param_file = "SOFI3D_%s.json"%expname
shellfilename = os.path.join(outpath,'runSOFI3D_%s.sh'%expname)


In [10]:
# GET THE DIFFERENT STRINGS - this will be updated in the future
default_strs = get_default_strings()

monitor_strs = get_monitor_str(tsnap_params, smgrm_ndt, expname, sbsmp_xyz=1)
t_str = get_time_str(dt, tdur)
mod_strs = get_subsurfmod_str(n_xzy, d_xzy,expname)
bndry_str = get_boundary_str(fs=False, npad=0,cpml=False)
src_str = get_source_str()

write_SOFIjsonParams = write_SOFIjsonParams(default_strs, 
                                            monitor_strs,  
                                            t_str, 
                                            mod_strs, 
                                            src_str, 
                                            bndry_str, 
                                            SOFIjsonfilename)
runscript = write_SOFIrunScript(sofimaster, 
                                num_procs, 
                                sofi_param_file, 
                                shellfilename)

In [11]:
mod_rho.transpose(0,2,1).shape

(20, 10, 10)